<a href="https://colab.research.google.com/github/vy-h-n/data_analytics/blob/main/Legal%20Processing%20Time%20for%20Food%20Poisoning%20Claims.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Background**

This notebook cleans and validates data using Pandas, queries data using SQLite3 and visualises data using Plotly on Google Colab.

Vivendo is a fast food chain in Brazil with over 200 outlets. Customers often claim compensation from the company for food poisoning. The legal team processes these claims. The legal team has offices in four locations. The legal team wants to improve how long it takes to reply to customers and close claims. The head of the legal department wants a report on how each location differs in the time it takes to close claims.

In [1]:
#Import libraries
import sqlite3
import pandas as pd
import numpy as np
import plotly.express as px

# Upload data set and create a Pandas data frame

**Data description**
* **claim_id:** Nominal. The unique identifier of the claim. Missing values are not possible due to the database structure.
* **time_to_close:** Discrete and positive. The number of days to close the claim.
* **claim_amount:** Continuous. The initial claim requested in the currency of Brazil, rounded to 2 decimal places.
* **amount_paid:** Continuous. Final amount paid. In the currency of Brazil. Rounded to 2 decimal places. Replaced missing values with the overall median amount paid.
* **location:** Nominal. Location of the claim, one of “RECIFE”, “SAO LUIS”, “FORTALEZA”, or “NATAL”. No missing values.
* **individuals_on_claim:** Discrete. Number of individuals on this claim. No null value. Minimum 1 person.
* **linked_cases:** Boolean. Whether this claim is linked to other cases. Either TRUE or FALSE. Replace missing values with FALSE.
* **cause:** Nominal. Cause of the food poisoning. One of “vegetable”, “meat” or “unknown”.

In [2]:
#Get data set from DataCamp
!wget https://s3.amazonaws.com/talent-assets.datacamp.com/food_claims_2212.csv

--2024-01-27 01:05:08--  https://s3.amazonaws.com/talent-assets.datacamp.com/food_claims_2212.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.73.94, 52.217.112.64, 54.231.195.96, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.73.94|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 104315 (102K) [text/csv]
Saving to: ‘food_claims_2212.csv’

food_claims_2212.cs 100%[===================>] 101.87K   594KB/s    in 0.2s    

2024-01-27 01:05:08 (594 KB/s) - ‘food_claims_2212.csv’ saved [104315/104315]



In [3]:
#Read the CSV data into a DataFrame
df = pd.read_csv('food_claims_2212.csv')

#Display the first few rows of the DataFrame to verify the data
df.head()

,claim_id,time_to_close,claim_amount,amount_paid,location,individuals_on_claim,linked_cases,cause
0,1,317,R$ 74474.55,51231.37,RECIFE,15,False,unknown
1,2,195,R$ 52137.83,42111.30,FORTALEZA,12,True,unknown
2,3,183,R$ 24447.2,23986.30,SAO LUIS,10,True,meat
3,4,186,R$ 29006.28,27942.72,FORTALEZA,11,False,meat
4,5,138,R$ 19520.6,16251.06,RECIFE,11,False,vegetable


The data set contains 1 table, which consists of 8 columns and 2000 rows including some null values.

In [4]:
#Check data type of each column in the data frame
data_types = df.dtypes
data_types

claim_id                  int64
time_to_close             int64
claim_amount             object
amount_paid             float64
location                 object
individuals_on_claim      int64
linked_cases             object
cause                    object
dtype: object

# TASK 1: Data cleaning and validation
*Data cleaning and validation is needed since data is not consistent with the requirements of DataCamp's data description.*

In [5]:
df["claim_id"].isnull().sum() #Check null value
df["claim_id"].nunique() #Check unique values

df["claim_id"] = df["claim_id"].astype(str) #Cast integers in column "claim_id" into strings

df["claim_id"]

0          1
1          2
2          3
3          4
4          5
        ... 
1995    1996
1996    1997
1997    1998
1998    1999
1999    2000
Name: claim_id, Length: 2000, dtype: object

**Claim ID:** There were 2000 unique values. No missing value. The data type of "claim_id" (descrete) were changed to string.

In [6]:
df["time_to_close"].isnull().sum() #Check null value
all_positive = (df["time_to_close"] > 0 ).all() #Check if all values are larger than 0
all_positive

True

**Time to close:** The values of the column 'time_to_close' were all positive discrete numbers. No missing values. No changes were made to this column.

In [7]:
df["claim_amount"].isnull().sum() #Check null value

#Strip the "R$ " part and keep only the numeric part
#Cast data to real number
#Round data to 2 decimal places
df["claim_amount"] = df["claim_amount"].str.strip("R$ ").astype(float).round(2)
df["claim_amount"]

0       74474.55
1       52137.83
2       24447.20
3       29006.28
4       19520.60
          ...   
1995    28982.30
1996     5188.44
1997    11975.85
1998    23516.28
1999     8051.40
Name: claim_amount, Length: 2000, dtype: float64

**Claim amount**: There were 2000 values in the 'claim_amount' column, no value was missing.

The type of data was text, not continuous data (as given in the description). Thus, for each row in column 'claim_amount', a number which is part of the text was extracted, casted to real number, and then rounded to 2 decimal place.

In [8]:
df["amount_paid"] #Check data type
df["amount_paid"].isnull().sum() #Check if there is null value
median = df["amount_paid"].median() #Median is calculated excluding null values
amount_paid = df["amount_paid"].fillna(median) #Replace null with the median

#Double check by identifying values that was replaced by the median
replaced_null = amount_paid.loc[amount_paid == median]
replaced_null

#Update new change
df["amount_paid"] = amount_paid
df["amount_paid"]

0       51231.37
1       42111.30
2       23986.30
3       27942.72
4       16251.06
          ...   
1995    24265.02
1996     4772.77
1997    10087.81
1998    23310.24
1999     6417.92
Name: amount_paid, Length: 2000, dtype: float64

**Amount paid**: Continuous. All valid values had 2 decimal digits. There were 1964 values in the column 'amount_paid', missing 36 values.

Missing values were replaced by the median calculated from valid values in the column 'claim_amount'.

In [9]:
#Validate data in column "location"
df["location"].unique()
df["location"].isnull().sum()

0

**Location:** Nonimal. There were 2000 values and no missing value. Location in the column was one of “RECIFE”, “SAO LUIS”, “FORTALEZA”, or “NATAL”. No changes were made.

In [10]:
df["individuals_on_claim"].isnull().sum() #Check if there is null value
df["individuals_on_claim"].dtypes #Check data type
all_positive = (df["individuals_on_claim"] > 0 ).all() #Check if all values are positve
all_positive #Print result

True

**Individuals on claim:** Data in this column was discrete. All values are not null and greater than or equal to 1. No changes were made.

In [11]:
df["linked_cases"].isnull().sum() #Look for null values
linked_cases = df["linked_cases"].fillna(False) #Replace null with boolean "False"
linked_cases.unique() #Check unique values
print(linked_cases.dtypes == bool) #Check data type of data frame "linked_cases"

#Update new changes
df["linked_cases"] = linked_cases
df["linked_cases"]

True


0       False
1        True
2        True
3       False
4       False
        ...  
1995    False
1996     True
1997     True
1998    False
1999    False
Name: linked_cases, Length: 2000, dtype: bool

**Linked_cases:** Boolean. There were 26 missing values, which has been replaced by the boolean "False".

In [12]:
df["cause"].isnull().sum() #Check if there is null value
df["cause"].unique() #Check unique values

#Replace data to ensure consistency
replacement_dict = {
    "VEGETABLES": "vegetable",
    "Meat": "meat",
    " meat": "meat"
}

df["cause"] = df["cause"].replace(replacement_dict, regex=True)

**Cause:** Nominal. There no null values. There were inconsistency in capitalization for the text in this column. For example, "meat", " meat" and "Meat", or "vegetable" and "VEGETABLES" exist in the records. Text records are standardised to ensure consistency.

# Connect to SQLite database first

In [13]:
#Connect to a SQLite database (it will be created if it doesn't exist)
conn = sqlite3.connect('food_claims')
cursor = conn.cursor()
print("SQL database connected")

SQL database connected


In [14]:
#Store the DataFrame in a SQLite table named 'food_claims'
df.to_sql('food_claims', conn, if_exists='replace', index=False)

2000

In [15]:
#Double check, in this SQLite database we only have 1 table named 'food_claims'
table_info = pd.read_sql("SELECT * FROM sqlite_master WHERE type='table'", conn)
table_info

,type,name,tbl_name,rootpage,sql
0,table,food_claims,food_claims,2,"CREATE TABLE ""food_claims"" (\n""claim_id"" TEXT,..."


In [16]:
#Check column name and data type from the SQLite database
pd.read_sql("""PRAGMA table_info(food_claims);""",
            conn)

,cid,name,type,notnull,dflt_value,pk
0,0,claim_id,TEXT,0,None,0
1,1,time_to_close,INTEGER,0,None,0
2,2,claim_amount,REAL,0,None,0
3,3,amount_paid,REAL,0,None,0
4,4,location,TEXT,0,None,0
5,5,individuals_on_claim,INTEGER,0,None,0
6,6,linked_cases,INTEGER,0,None,0
7,7,cause,TEXT,0,None,0


In the SQLite table named "food_claims" we have 8 columns.

# TASK 2: Distribution of claim ID in each location

In [17]:
#Create data
query = """ SELECT COUNT(claim_id) AS cclaim_id, location
            FROM food_claims
            GROUP BY location
            ORDER BY cclaim_id;
"""
bar_chart = pd.read_sql(query, conn)

#Create a bar chart
fig = px.bar(bar_chart, x='location', y='cclaim_id', title='Distribution of Claim ID per Location')

#Set new labels for the x-axis and y-axis directly
fig.update_xaxes(title_text='Location')
fig.update_yaxes(title_text='Claim ID')

#Show the chart
fig.show()

The office located in Recife receives the highest number of claims from customers (885 claims), followed by Sao Luis (517 claims), then Fortaleza (311 claims) and lastly Natal (287 claims). The disparity between the highest (in Recife) and the lowest (in Natal) is approximately three times greater.

# TASK 3: Distribution of time to close for all claims

In [18]:
#Describe data in the column "time_to_close"
df["time_to_close"].describe()

count    2000.00000
mean      185.56800
std        49.16339
min        76.00000
25%       158.00000
50%       179.00000
75%       204.00000
max       518.00000
Name: time_to_close, dtype: float64

In [19]:
#Create a histogram plot
fig = px.histogram(df, x='time_to_close', nbins=10, title='Distribution of Time to Close')
fig.update_xaxes(title='Time to Close')
fig.update_yaxes(title='Frequency')
fig.show()

The average time taken for the legal team to close a claim is around 186 minutes. The quickest time to resolve a claim is 76 minutes, and the slowest takes 518 minutes which only happens in extreme cases. Most of the claims take 150-199 minutes to be finalised.

#TASK 4: Relationship between time to close and location

In [20]:
# Create a box plot
fig = px.box(df, x='location', y='time_to_close', title='Time to Close Distribution by Location')

# Customize the plot
fig.update_xaxes(title='Location')
fig.update_yaxes(title='Time to Close')

# Show the plot
fig.show()

Outliers from the box and whisker plot are to be removed for better comparision.

In [21]:
#Calculate the IQR for 'time_to_close'
Q1 = df['time_to_close'].quantile(0.25)
Q3 = df['time_to_close'].quantile(0.75)
IQR = Q3 - Q1

#Define the lower and upper bounds to identify outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

#Filter the DataFrame to remove outliers
filtered_df = df[(df['time_to_close'] >= lower_bound) & (df['time_to_close'] <= upper_bound)]

#Create a box plot with the filtered data
fig = px.box(filtered_df, x='location', y='time_to_close', title='Time to Close Distribution by Location (Outliers Removed)')
fig.update_xaxes(title='Location')
fig.update_yaxes(title='Time to Close')

# Show the plot
fig.show()

Location is not correlated to time taken to resolve a claim. Among the locations, no significant difference in terms of the median and the range of time taken to close a claim handled by the legal team.